## Bedrock model integration with Langchain Agents

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

In this notebook, we will demonstrate the use `plan-and-execute` agents along with `Zero-shot ReAct` which is an action agent and uses the [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) framework to select the appropriate tool based exclusively on the tool's description. It requires you provide the description of each tool. 

## Architecture
![](./images/arch-agents.png)

In [8]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=852d8d928d06ddf8f14fd523fccf5ac923e69f48674c64bafbc322daab4bb041
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [9]:
import json
import os
import sys
import boto3

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ['SERPAPI_API_KEY'] = "<YOUR_SERP_API_KEY>" # Required for the search tool

In [26]:
# model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000} # for nthropic.claude-instant-v1
model_parameter = {"temperature": 0.0, "topP": .5, "maxTokenCount": 2000} # for amazon.titan-text-express-v1

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [27]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

Load two unique LLM objects with unique model parameters, one for the agent, and the other for the math chain.

A unique llm with stop sequences for the math chain helps prevent Claude from being overly verbose while running the math chain.

Additionally, we'll adapt the default template to better fit Claude, as the default Langchain templates are not well fit by default, then append the newly constructed tool to our list of tools.

In [28]:
# react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)
# math_chain_llm = Bedrock(model_id="anthropic.claude-instant-v1",
#                          model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

#Below two commands are added because access to claude is not there 
react_agent_llm = Bedrock(model_id="amazon.titan-text-express-v1", model_kwargs=model_parameter)
math_chain_llm = Bedrock(model_id="amazon.titan-text-express-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools = load_tools(["wikipedia"], llm=react_agent_llm)

llm_math_chain =  LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [35]:
react_agent = initialize_agent(tools, 
                               react_agent_llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                               handle_parsing_errors=True
                               )

In [36]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Wikipedia", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

In [37]:
react_agent.agent.llm_chain.prompt.template=prompt_template

In [38]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"

In [39]:
react_agent(question)



> Entering new AgentExecutor chain...
Amazon SageMaker is a fully managed machine learning service that helps you prepare, build, train, and deploy high-quality machine learning models quickly.

The launch year of Amazon SageMaker multiplied by 2 is 2022.

Would you like to know more about Amazon SageMaker?
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought: I now know the final answer
Final Answer: 2022

> Finished chain.


{'input': 'What is Amazon SageMaker? What is the launch year multiplied by 2',
 'output': '2022'}

## Generating the tool code
Could you use GenAI to author the code in the tools? Sure! With caution, there is some variation in the code and 
may not be safe for execution without a human review.

The following example uses the code generated by Claude to generate the tool. It replaces the EC2Search tool above with the boto python code.

In [52]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

boto3_bedrock = boto3.client(service_name='bedrock-runtime',
                             region_name=os.environ["AWS_DEFAULT_REGION"])

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances using boto3 resource that have a tag equal to the tagname parameter. 

return the code inside <code></code>.
Assistant:"""

# body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500}) # for claude
body = json.dumps({"inputText": prompt_data}) # for amazon titan
# modelId = "anthropic.claude-instant-v1"  # for claude
modelId = "amazon.titan-text-express-v1" # for amazon titan
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

# tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
tree = ET.ElementTree(ET.fromstring(response_body.get("results")[0].get("outputText")))
# python_code = tree.getroot().text

# display(Markdown(f'```{python_code}```'))

# exec(python_code)

# tools = [
#     Tool.from_function(
#         func=list_tagged_instances,
#         name="List",
#         description="Queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter."
#     ),
# ]

TypeError: a bytes-like object is required, not 'list'

In [54]:
tree = ET.ElementTree(ET.fromstring(response_body.get("results")[0].get("outputText")))

ParseError: syntax error: line 1, column 1 (<string>)

In [48]:
response_body

{'inputTextTokenCount': 88,
 'results': [{'tokenCount': 128,
   'outputText': " Sure! Here's the python function named list_tagged_instances with one parameter named tagname:\n\n```python\nimport boto3\n\ndef list_tagged_instances(tagname):\n    try:\n        # Create a resource object for EC2 instances\n        ec2 = boto3.resource('ec2')\n\n        # Filter instances based on the specified tag\n        instances = ec2.instances.filter(Filters=[{'Name': 'tag-key', 'Values': [tagname]}])\n\n        # Return a list of EC2 instances with the specified tag\n        return instances['Inst",
   'completionReason': 'LENGTH'}]}

## Executing the code
If the code above looks reasonable, we can use it to run our agent.

**Note - for this to work, you need an EC2 instance in this account with a tag named 'delete' and any value in the tag. It is case senstitive, so prior to executing this, create an EC2 instance and set it to the stopped state.**

In [12]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the tag delete.
Assistant:"""

result = react_agent.run(question)

print(f"{result}")



> Entering new AgentExecutor chain...
 I need to query EC2 for instances tagged 'delete'
Action: List
Action Input: delete
Observation: [ec2.Instance(id='i-02c97adedeab52621')]
Thought: I now have the instance ID of the instance tagged 'delete'
Final Answer: i-02c97adedeab52621

> Finished chain.
i-02c97adedeab52621


## Another variation - Search for an instance that does not exist
The prompt changes to search for EC2 instances with a tag that does not exist, so no instances should be returned.

In [13]:
question = """Human: Please list my EC2 instances with the tag doesnotexist.
Assistant:"""

result = react_agent(question)

print(f"{result}")



> Entering new AgentExecutor chain...
 I need to check if there are any EC2 instances with the tag "doesnotexist"
Action: List
Action Input: doesnotexist
Observation: []
Thought: The list action returned an empty list, so there are no EC2 instances with that tag.
Final Answer: There are no EC2 instances with the tag "doesnotexist".

> Finished chain.
{'input': 'Human: Please list my EC2 instances with the tag doesnotexist.\nAssistant:', 'output': 'There are no EC2 instances with the tag "doesnotexist".'}


## Database Tools
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

### SQL Database Agent
Langachain has a SQL Database agent for demonstrating how to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/integrations/toolkits/sql_database

The agent will load the schema of the DB into context and generate SQL statements based on natural language questions. The SQL statement is then executed against the database and the results returned.

### Data Agents
While the SQL Database agent is useful for data exploration and generating queries, there are also cases where you want to 
For specific entities, a tool can be created to pull data from the database to provide context next steps in the prompt.

The following example will simulate a DB query for a customer in the customer table. Replace this code with a lookup ib DynamoDB or a relation database.

In [14]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [15]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """\n\nHuman: write one sentence summary about the information you know about the customer with an id of 2.

\n\nAssistant: Here is the one sentence summary: """

result = react_agent.run(question)

print(f"{result}")



> Entering new AgentExecutor chain...
 I need to lookup the customer with id 2
Action: CustomerLookup
Action Input: 2search by customer 2
found customer 2 {'id': 2, 'first_name': 'Jane', 'last_name': 'Smith', 'age': 27, 'postal_code': '12345'}

Observation: {'id': 2, 'first_name': 'Jane', 'last_name': 'Smith', 'age': 27, 'postal_code': '12345'}
Thought: I now know the details about the customer with id 2
Final Answer: The customer with id 2 is Jane Smith who is 27 years old and has a postal code of 12345.

> Finished chain.
The customer with id 2 is Jane Smith who is 27 years old and has a postal code of 12345.
